# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [ ]:
import sys
sys.path.append('../')
from tokenizer import tokenizer
from vectorizer import vectorizer

import sklearn.model_selection
import numpy as np
import pandas as pd
import sklearn as sk

In [ ]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [ ]:
# FOR TESTING : only select first 20000 samples
X, y = X[:20000], y[:20000]

In [ ]:
tokenized_documents = tokenizer(X)

In [ ]:
X, vect = vectorizer(tokenized_documents)

In [ ]:
# most frequent words
print("Top 10 most frequent words in the dataset")
print(vect.get_feature_names_out()[:10])

# least frequent words
print("Top 10 least frequent words in the dataset")
print(vect.get_feature_names_out()[-10:])

In [ ]:
assert X.shape[0] == y.shape[0], "Le nombre d'échantillons dans X et y est différent."

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

# Naive Bayes Classifier
### Model starts here

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Multinomial Naive Bayes
nb = MultinomialNB()
naive_bayes = nb.fit(X_train, y_train)
predicted = naive_bayes.predict(X_test)

# Metrics
from sklearn import metrics
print(metrics.confusion_matrix(y_test, predicted))
print(metrics.classification_report(y_test, predicted))

In [ ]:
example = '''
awful
'''

# tokenize and vectorize it, then try to predict
test = tokenizer([example])
test_tfidf = vect.transform(test)
print(naive_bayes.predict(test_tfidf))